# Ordering InSAR Products Using the HyP3 Python SDK

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ASFHyP3/hyp3-sdk/main?filepath=docs%2Fsdk_example.ipynb)





## Using the HyP3 SDK for Python

HyP3's Python SDK `hyp3_sdk` provides a convenience wrapper around the HyP3 API and HyP3 jobs.


The HyP3 SDK can be installed using [Anaconda/Miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html#anaconda-or-miniconda)
 (recommended) via [`conda`](https://anaconda.org/conda-forge/hyp3_sdk):

```
conda install -c conda-forge hyp3_sdk
```

Or using [`pip`](https://pypi.org/project/hyp3-sdk/):

```
python -m pip install hyp3_sdk
```

Full documentation of the SDK can be found in the [HyP3 documentation](https://hyp3-docs.asf.alaska.edu/using/sdk/).

### Other recommended packages

We also recommend installing the `asf_search` Python package for performing searches of the ASF catalog. The ASF Search
Python package can be installed using [Anaconda/Miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html#anaconda-or-miniconda)
(recommended) via [`conda`](https://anaconda.org/conda-forge/asf_search):

```
conda install -c conda-forge asf_search
```

Or using [`pip`](https://pypi.org/project/asf-search):

```
python -m pip install asf_search
```

Full documentation of `asf_search` can be found in the [ASF search documentation](https://docs.asf.alaska.edu/asf_search/basics).

In [ ]:
# initial setup
import asf_search as asf
import hyp3_sdk as sdk

In [ ]:
# use .netrc
hyp3 = sdk.HyP3()

In [ ]:
# or enter your credentials
hyp3 = sdk.HyP3(prompt=True)

## Submitting jobs

The SDK provides a submit method for [all supported job types](https://hyp3-docs.asf.alaska.edu/products/), but for the purposes of this tutorial, we will focus on submitting a [InSAR job](https://hyp3-docs.asf.alaska.edu/products/#insar).

### Submitting Sentinel-1 InSAR jobs

The SDK can submit Sentinel-1 Interferometric Synthetic Aperture Radar (InSAR) jobs which processes
reference and secondary granule pairs.

For a particular reference granule, we may want to use the nearest and next-nearest temporal neighbor granules as secondary
scenes. To programmatically find our secondary granules for a reference granule, We'll define a `get_nearest_neighbors`
function that uses the [baseline stack](https://docs.asf.alaska.edu/asf_search/ASFProduct/#stack) method from `asf_search`:

In [ ]:
from typing import Optional

def get_nearest_neighbors(granule: str, max_neighbors: Optional[int] = None) -> asf.ASFSearchResults:
    granule = asf.granule_search(granule)[-1]
    stack = reversed([item for item in granule.stack() if item.properties['temporalBaseline'] < 0])
    return asf.ASFSearchResults(stack)[:max_neighbors]

Now, using the example granule list for our RTC jobs as the reference scenes, we can find their nearest and next-nearest neighbor granules, and submit them
as pairs for InSAR processing.

In [ ]:
from tqdm.auto import tqdm  # For a nice progress bar: https://github.com/tqdm/tqdm#ipython-jupyter-integration

insar_jobs = sdk.Batch()
for reference in tqdm(granules):
    neighbors = get_nearest_neighbors(reference, max_neighbors=2)
    for secondary in neighbors:
        insar_jobs += hyp3.submit_insar_job(reference, secondary.properties['sceneName'], name='insar-example')
print(insar_jobs)

Like RTC jobs, `HyP3.submit_insar_job` accepts
[keyword arguments](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_insar_job)
to customize the InSAR products to your application.

## Monitoring jobs

One jobs are submitted, you can either watch the jobs until they finish

In [ ]:
rtc_jobs = hyp3.watch(insar_jobs)

which will require you to keep the cell/terminal running, or you can come back later and search for jobs

In [ ]:
insar_jobs = hyp3.find_jobs(name='insar-example')
insar_jobs = hyp3.watch(insar_jobs)

### Downloading files

Batches are collections of jobs. They provide a snapshot of the job status when the job was created or last
refreshed. To get updated information on a batch

In [ ]:
print(insar_jobs)
insar_jobs = hyp3.refresh(insar_jobs)
print(insar_jobs)

`hyp3.watch()` will return a refreshed batch once every job in the batch has completed.

and filter jobs by status

In [ ]:
succeeded_jobs = insar_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = insar_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

You can download the files for all successful jobs

In [ ]:
file_list = succeeded_jobs.download_files()

*Note: only succeeded jobs will have files to download.*

### Image Files

All of the main InSAR product files are 32-bit floating-point single-band GeoTIFFs. To learn more about the rasters included in the product package, refer to the [Exploring Sentinel-1 InSAR StoryMap](https://storymaps.arcgis.com/stories/8be186e4125741518118d0102e6835e5 "Exploring Sentinel-1 InSAR StoryMap https://arcg.is/11DaW90" ) tutorial.

- The *amplitude* image is the calibrated radiometric backscatter from the reference granule in sigma-nought power. The image is terrain corrected using a geometric correction, but not radiometrically corrected.
- The *coherence* file pixel values range from 0.0 to 1.0, with 0.0 being completely non-coherent and 1.0 being perfectly coherent.
- The *unwrapped phase* file shows the results of the phase unwrapping process. Negative values indicate movement towards the sensor, and positive values indicate movement away from the sensor. This is the main interferogram output.
- The *wrapped phase* file indicates the interferogram phase after applying the adaptive filter immediately before unwrapping. Values range from negative pi to positive pi. *(optional)*
- The *line-of-sight displacement* file indicates the displacement in meters along the look direction of the sensor. The sign is opposite to that of the unwrapped phase: positive values indicate motion towards the sensor and negative values indicate motion away from the sensor. *(optional)*
- The *vertical displacement* is generated from the line of sight displacement values, and makes the assumption that deformation only occurs in the vertical direction. Note that this assumption may not hold true in cases where the deformation also has a horizontal component. Positive values indicate uplift, and negative values indicate subsidence. *(optional)*
- The *look vectors* theta (θ) and phi (φ) describe the elevation and orientation angles of the sensor's look direction. *(optional)*
- The *incidence angle* maps indicate the angle between the incident signal and the surface normal of either the terrain (local incidence angle) or the ellipsoid (ellipsoid incidence angle). *(optional)*
- The *DEM* file gives the local terrain heights in meters, with a geoid correction applied. *(optional)*
- The *water mask* file indicates coastal waters and large inland waterbodies. Pixel values of 1 indicate land and 0 indicate water. This file is in 8-bit unsigned integer format.

If the **water mask** option is selected, the water mask is applied prior to phase unwrapping to exclude water pixels from the process. The water mask is generated using the [GSHHG](http://www.soest.hawaii.edu/wessel/gshhg "http://www.soest.hawaii.edu/wessel/gshhg/land" ){target=_blank} dataset. To compile the reference shapefile, the full-resolution L1 dataset (boundary between land and ocean) and L5 dataset (boundary between Antarctic ice and ocean) were combined. The L3 dataset (boundary between islands and the lakes they are within) was removed from the L2 dataset (boundary between lakes and land), and this combined dataset was removed from the combined L1/L5 dataset. The portion of the shapefile covering the scene is converted to a raster for inclusion in the phase unwrapping mask during InSAR processing. The GSHHG dataset was last updated in 2017, so there may be discrepancies where shorelines have changed. Visit our [InSAR Water Masking Tutorial](https://storymaps.arcgis.com/stories/485916be1b1d46889aa436794b5633cb "InSAR Water Masking StoryMap" ){target=_blank} for more information about water masking.

**Browse images** are included for the wrapped (color_phase) and unwrapped (unw_phase) phase files, which are in PNG format and are each 2048 pixels wide. The browse images are displayed using a cyclic color ramp to generate fringes.

- Each fringe in a wrapped (color_phase) browse image represents a 2-pi phase difference, and the line-of-sight displacement for each fringe is equivalent to half the wavelength of the sensor. The wavelength of Sentinel-1 is about 5.6 cm, so each 2-pi fringe represents a line-of-sight displacement of about 2.8 cm.
- Each fringe in an unwrapped (unw_phase) browse image represents a phase difference of 6 pi. Because each 2-pi difference is equivalent to half the wavelength of the sensor, each 6-pi fringe represents about 8.3 cm of line-of-sight displacement for these Sentinel-1 products.

**KMZ files** are included for the wrapped (color_phase) and unwrapped (unw_phase) phase images, which allow users to view the outputs in Google Earth or other platforms that support kmz files.

The tags and extensions used and example file names for each raster are listed in Table 1 below.

{% set base_name = 'S1AB<wbr>_20171111T150004<wbr>_20171117T145926<wbr>_VVP006<wbr>_INT80<wbr>_G<wbr>_ueF<wbr>_4D09<wbr>' %}

| Extension          | Description                      | Example                           |
|--------------------|----------------------------------|-----------------------------------|
| _amp.tif           | Amplitude                        | {{ base_name }}_amp.tif           |
| _corr.tif          | Normalized coherence file        | {{ base_name }}_corr.tif          |
| _unw_phase.tif     | Unwrapped geocoded interferogram | {{ base_name }}_unw_phase.tif     |
| _wrapped_phase.tif | Wrapped geocoded interferogram   | {{ base_name }}_wrapped_phase.tif |
| _los_disp.tif      | Line-of-sight displacement       | {{ base_name }}_los_disp.tif      |
| _vert_disp.tif     | Vertical displacement            | {{ base_name }}_vert_disp.tif     |
| _lv_phi.tif        | Look vector φ (orientation)      | {{ base_name }}_lv_phi.tif        |
| _lv_theta.tif      | Look vector θ (elevation)        | {{ base_name }}_lv_theta.tif      |
| _dem.tif           | Digital elevation model          | {{ base_name }}_dem.tif           |
| _inc_map_ell.tif   | Ellipsoid incidence angle        | {{ base_name }}_inc_map_ell.tif   |
| _inc_map.tif       | Local incidence angle            | {{ base_name }}_inc_map.tif       |
| _water_mask.tif    | Water mask                       | {{ base_name }}_water_mask.tif    |
| _color_phase.kmz   | Wrapped phase kmz file           | {{ base_name }}_color_phase.kmz   |
| _unw_phase.kmz     | Unwrapped phase kmz file         | {{ base_name }}_unw_phase.kmz     |
| _color_phase.png   | Wrapped phase browse image       | {{ base_name }}_color_phase.png   |
| _unw_phase.png     | Unwrapped phase browse image     | {{ base_name }}_unw_phase.png     |

*Table 1: Image files in product package*

### Metadata Files

The product package also includes a number of metadata files.

| Extension      | Description                                     | Example                                 |
|----------------|-------------------------------------------------|-----------------------------------------|
| .README.md.txt | Main README file for GAMMA InSAR                | {{ base_name }}.README.md.txt           |
| .txt           | Parameters and metadata for the InSAR pair      | {{ base_name }}.txt                     |
| .tif.xml       | ArcGIS compliant XML metadata for GeoTIFF files | {{ base_name }}_unw_phase.tif.xml       |
| .png.xml       | ArcGIS compliant XML metadata for PNG files     | {{ base_name }}_color_phase.png.xml     |
| .png.aux.xml   | Geolocation information for png browse images   | {{ base_name }}_color_phase.png.aux.xml |

*Table 3: Metadata files in product package*